# Dropout Model

## Step 1: Mount Google Drive and Download Images

In [0]:
pip install mxnet

    100% |████████████████████████████████| 28.4MB 885kB/s 
    100% |████████████████████████████████| 61kB 16.7MB/s 
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 1.0.0 has requirement requests~=2.18.0, but you'll have requests 2.21.0 which is incompatible.
fastai 1.0.51 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4


In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import h5py
from matplotlib import pyplot as plt
import numpy as np

In [0]:
f = h5py.File('/content/drive/My Drive/images.h5', 'r')

images_train = f['Train/images'][...]
labels_train = f['Train/labels'][...]

images_test = f['Test/images'][...]
labels_test = f['Test/labels'][...]

f.close()
type(images_test)

## Step 5: Flatten the Images

In [0]:
num_train = images_train.shape[0]
images_train_flatten = images_train.flatten().reshape(num_train, 100*100*3)
print("Number Training Images: ", num_train)
print("Shape of Flattened Training Images Array: ", images_train_flatten.shape)

num_test = images_test.shape[0]
images_test_flatten = images_test.flatten().reshape(num_test, 100*100*3)
print("Number Testing Images: ", num_test)
print("Shape of Flattened Testing Images Array: ", images_test_flatten.shape)

Number Training Images:  4405
Shape of Flattened Training Images Array:  (4405, 30000)
Number Testing Images:  1617
Shape of Flattened Testing Images Array:  (1617, 30000)


## Step 6: Standardize the Data

Here we imported the alternate model

In [0]:
from __future__ import print_function
import mxnet as mx
import numpy as np
from mxnet import nd, autograd, gluon
mx.random.seed(1)
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()

The function we would use would need to get the data as float64 type and in smaller chunks as opposed to all at once, so we applied a transformation functiona and loaded it into a Mxnet dataloader.

In [0]:
train_set_x = images_train_flatten/255.
test_set_x = images_test_flatten/255.
def transform(data):
  return data.astype(np.float64)

#print(train_set_x)
#type(train_set_x)
#print(train_set_x[0:3,0:3])
traindataset = mx.gluon.data.dataset.ArrayDataset(transform(train_set_x),transform(labels_train))
testdataset = mx.gluon.data.dataset.ArrayDataset(transform(test_set_x),transform(labels_test))

In [0]:
#def transform(data, label):
   # return data.astype(np.float64), label.astype(np.float64)
#ctrain=transform()  
#ctrain=traindataset.transform(transform,lazy=False)
#ctest=testdataset.transform(transform,lazy=False)
train=mx.gluon.data.DataLoader(traindataset, batch_size=25, shuffle=True) #sampler=None, last_batch=None, batch_sampler=None)
test=mx.gluon.data.DataLoader(testdataset, batch_size=25, shuffle=False) #sampler=None, last_batch=None, batch_sampler=None)
#print(ctrain)

In [0]:
#batch_size= 64
#mnist = mx.test_utils.get_mnist()
#def transform(data, label):
#    return data.astype(np.float32)/255, label.astype(np.float32)
#train_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True, transform=transform), batch_size, shuffle=True)
#print(gluon.data.vision.MNIST(train=True, transform=transform))
#print(mnist)
#np.shape(train_data)

## Step 7: Assembling the Model

###This is the link for the method below:

https://gluon.mxnet.io/chapter03_deep-neural-networks/mlp-dropout-scratch.html

Let's start coding pieces of the model.

In [0]:
W1 = nd.random_normal(shape=(784,256), ctx=ctx, dtype=np.float64) *.01
b1 = nd.random_normal(shape=256, ctx=ctx, dtype=np.float64) * .01

W2 = nd.random_normal(shape=(256,128), ctx=ctx, dtype=np.float64) *.01
b2 = nd.random_normal(shape=128, ctx=ctx, dtype=np.float64) * .01

W3 = nd.random_normal(shape=(128,10), ctx=ctx, dtype=np.float64) *.01
b3 = nd.random_normal(shape=10, ctx=ctx, dtype=np.float64) *.01

params = [W1, b1, W2, b2, W3, b3]

In [0]:
W1.dtype

numpy.float64

###Space for gradients

In [0]:
for param in params:
    param.attach_grad()
    
def relu(X):
  return nd.maximum(X, 0)

###Dropout Function

In [0]:

def dropout(X, drop_probability):
    keep_probability = 1 - drop_probability
    mask = nd.random_uniform(0, 1.0, X.shape, ctx=X.context) < keep_probability
    #############################
    #  Avoid division by 0 when scaling
    #############################
    if keep_probability > 0.0:
        scale = (1/keep_probability)
    else:
        scale = 0.0
    return mask * X * scale
  

A = nd.arange(20).reshape((5,4))
dropout(A, 0.0)
dropout(A, 0.5)
dropout(A, 1.0)


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
<NDArray 5x4 @cpu(0)>

###Softmax output and cross-entropy loss function

In [0]:
def softmax(y_linear):
    exp = nd.exp(y_linear-nd.max(y_linear))
    partition = nd.nansum(exp, axis=0, exclude=True).reshape((-1,1))
    return exp / partition
  
def softmax_cross_entropy(yhat_linear, y):
    return - nd.nansum(y * nd.log_softmax(yhat_linear), axis=0, exclude=True)

###Define Model

In [0]:
!pip install ipdb

  Stored in directory: /root/.cache/pip/wheels/59/24/91/695211bd228d40fb22dff0ce3f05ba41ab724ab771736233f3
Successfully built ipdb


In [0]:
import ipdb as debugger

In [0]:
def net(X, drop_prob=0.0):
    #######################
    #  Compute the first hidden layer
    #######################
    
    h1_linear = nd.dot(X, W1) + b1
    h1 = relu(h1_linear)
    debugger.set_trace()
    h1 = dropout(h1, drop_prob)

    #######################
    #  Compute the second hidden layer
    #######################
    h2_linear = nd.dot(h1, W2) + b2
    h2 = relu(h2_linear)
    h2 = dropout(h2, drop_prob)

    #######################
    #  Compute the output layer.
    #  We will omit the softmax function here
    #  because it will be applied
    #  in the softmax_cross_entropy loss
    #######################
    yhat_linear = nd.dot(h2, W3) + b3
    return yhat_linear

###Optimizer

In [0]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

###Evaluation Metric

In [0]:
def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx).reshape((-1,784))
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        numerator += nd.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator / denominator).asscalar()

###Training Loop

Here is where we encountered problems and got stuck. Despite the transformations above, the coded model switched between data types several times in layers coded deeper than what we can see here.  As a result, we were unable to determine the proper input datatypes and recieved errors when we tried to run the training loop.

In [0]:
epochs = 10
moving_loss = 0.
learning_rate = .001

for e in range(epochs):
    for i, (data, label) in enumerate(train): 
        data = data.as_in_context(ctx).reshape((-1,784))
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 10)
        with autograd.record():
            ################################
            #   Drop out 50% of hidden activations on the forward pass
            ################################
            output = net(data, drop_prob=.5)
            loss = softmax_cross_entropy(output, label_one_hot)
        loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if i == 0:
            moving_loss = nd.mean(loss).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(loss).asscalar()

    test_accuracy = evaluate_accuracy(test, net)
    train_accuracy = evaluate_accuracy(train, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))

> <ipython-input-35-8cc056d65140>(9)net()
      8     debugger.set_trace()
----> 9     h1 = dropout(h1, drop_prob)
     10 

ipdb> type(h1)
<class 'mxnet.ndarray.ndarray.NDArray'>
ipdb> h1.dtype
<class 'numpy.float64'>
ipdb> drop_prob.dtype
*** AttributeError: 'float' object has no attribute 'dtype'
ipdb> q


BdbQuit: ignored